<a href="https://colab.research.google.com/github/fabriciosantana/nlp/blob/main/AKCIT_NLP_M6_Colab_Unidade_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Essa aula se baseia:



*   No artigo "*Distributed Representations of Words and Phrases
and their Compositionality*", de Mikolov et al. (2013). Clique [aqui](https://proceedings.neurips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf) para acesso.
*   Na documentação do word2vec da biblioteca [Gensim](https://radimrehurek.com/gensim/models/word2vec.html).
# Objetivos de Aprendizagem

Neste notebook mostramos como treinar um modelo Word2Vec do zero e como importar um modelo pré-treinado usando a biblioteca gensim.


## Princípios de Word Embeddings

Inicialmente é necessário instalar as bibliotecas keras, tensorflow, gensim e np_utils. Para instalar esses recursos você usará o comeando !pip install biblioteca


In [ ]:
!pip install keras
!pip install tensorflow
!pip install -U gensim
!pip install np_utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 567.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56437 sha256=88b5e83a6ca29bdb51a65a8ef7dc4495bde0d49e5e932e03db3506b0d3c9f8da
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
Successfully built np_utils


In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Embedding,Activation,Dropout,SimpleRNN,BatchNormalization,RNN,Flatten,Input,LSTM,Bidirectional
#from keras.utils.np_utils import to_categorical
from keras.utils import to_categorical
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import gensim
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Caso você queira treinar um modelo Word2vec do Zero, a biblioteca Gensim provê essa funcionalide:

Alguns comentários sobre o código a seguir:

1)Observe que estamos um usanddo um corpus já tratado, com palavras de interesse
(imagine que temos um corpus com várias sentenças já tokenizadas por espaços). Cada setença então está em um vetor e o corpus é um vetor de vetores. Entretanto, a entrada poderia vir de um arquivo no disco, da rede em tempo real, sem precisar carregar todo o seu corpus na RAM.

2) O Word2vec aceita vários parâmetros que afetam tanto a velocidade quanto a qualidade do treinamento.

3) Um deles é para podar o dicionário interno. Palavras que aparecem apenas uma ou duas vezes em um corpus de um bilhão de palavras são provavelmente erros de digitação desinteressantes e lixo. Além disso, não há dados suficientes para fazer qualquer treinamento significativo sobre essas palavras, então é melhor ignorá-las. Um valor razoável para min_count está entre 0-100, dependendo do tamanho do seu conjunto de dados.

4) O paralelismo de treinamento é tratado no número de workers, para acelerar o treinamento, default = 1 -- worker = no parallelization

5) vector_size (int, opcional) – Dimensionalidade dos vetores de palavras.

Aqui temos a lista completa de parâmetros e valores default.
https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec


6) Observe a variável model resultado:  Este objeto contém essencialmente o mapeamento entre palavras e embeddings. Após o treinamento, ele pode ser usado diretamente para consultar esses embeddings de várias maneiras.

In [ ]:
from gensim.models import Word2Vec
corpus = [["hello", "world","hi","earth","sunshine","law"],["cat", "say", "meow"], ["dog", "say", "woof"]]
model = Word2Vec(sentences=corpus, vector_size=5, window=5, min_count=1, workers=4)




Os vetores de palavras treinados são armazenados em uma instância KeyedVectors, como model.wv. O motivo para separar os vetores treinados em KeyedVectors é que se você não precisar mais do estado completo do modelo (não precisar continuar o treinamento), seu estado pode ser descartado, mantendo apenas os vetores e suas chaves adequadas.


Uma vez o modelo treinado podemos carregar esse modelo e realizar operações aritméticas entre termos:

In [ ]:
word_vectors = model.wv

Observe que o tamanho dos vetores obtidos é 5, definido na etapa de treinamento. Temos um vetor denso de tamanho 5 para cada palavra.

In [ ]:
len(word_vectors[0])

5

Aqui podemos verificar a operação de subtração dos vetores das palavras 'dog' e 'cat'.

In [ ]:
word_vectors['dog'] - word_vectors['cat']

array([ 0.00137478, -0.13207467, -0.22588612,  0.11582372, -0.22422102],
      dtype=float32)

Podemos ainda retreinar o modelo com novas palavras:

In [ ]:
model.train([["dear", "bear", "cream"]], total_examples=3, epochs=1)

(0, 3)

### Carregando Modelos Treinados em Outros Corpora

Existem diversos repositórios de *embeddings* pré-treinados para facilitar nossa vida. Algumas bibliotecas dispobilizam isso para nós usuários. Veja a senguir os modelos pré-treinados de *word embeddings* disponíveis na biblioteca gensim:

In [ ]:
import gensim.downloader

print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


Existem vários, mas vamos escolher um modelo treinado no google *news* com 300 dimensões e no idioma Inglês. Observe que esse processo pode ser um pouco demorado em função do tamanho deste modelo.

In [ ]:
word_vectors = gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


Agora, vamos usar este modelo? Quais são as palavras mais similares à palavra 'car'?

In [ ]:
word_vectors.most_similar('car')

[('vehicle', 0.7821096181869507),
 ('cars', 0.7423831224441528),
 ('SUV', 0.7160962224006653),
 ('minivan', 0.6907036900520325),
 ('truck', 0.6735789775848389),
 ('Car', 0.6677608489990234),
 ('Ford_Focus', 0.667320191860199),
 ('Honda_Civic', 0.6626849174499512),
 ('Jeep', 0.651133120059967),
 ('pickup_truck', 0.6441438794136047)]

Podemos realizar operaçõe aritméticas

In [ ]:
word_vectors['airplane']-word_vectors['flight']

array([ 2.53417969e-01, -2.23144531e-01,  3.80859375e-02,  2.54394531e-01,
       -1.92871094e-01, -1.28906250e-01, -3.61328125e-02,  2.16796875e-01,
        2.08007812e-01,  1.16699219e-01,  1.44042969e-01, -1.10351562e-01,
        9.17968750e-02, -2.11425781e-01, -6.05468750e-02,  1.19140625e-01,
       -1.39404297e-01,  1.58508301e-01,  2.34375000e-02, -4.05273438e-02,
        5.56640625e-02,  5.85021973e-02,  1.89697266e-01, -2.05078125e-02,
        4.58526611e-03, -7.69042969e-02,  6.83593750e-02, -4.39453125e-03,
       -7.51953125e-02, -1.25976562e-01, -2.45117188e-01, -1.58203125e-01,
       -1.97753906e-01,  2.93212891e-01,  2.24243164e-01,  1.37695312e-01,
        2.87658691e-01, -9.76562500e-02, -3.07617188e-02,  1.98242188e-01,
        7.30438232e-02,  5.85937500e-03, -1.48437500e-01,  7.95288086e-02,
       -5.29785156e-02,  1.97265625e-01, -1.09863281e-01, -2.58789062e-02,
        1.17187500e-02, -1.85546875e-01, -1.45019531e-01,  2.28515625e-01,
       -1.41601562e-02, -

Utilizando o método ```most_similar```, podemos retornar as palavras mais similares por meio de uma operação aritmética:
somam-se os vetores positivos e subtrai o vetor negativo. A partir do resultado, podemos obter as palavras mais similares comparando-se esse vetor resultante com os vetores das demais palavras do *word embedding* com base na similaridade cosseno. Abaixo as top 10 palavras mais similares são mostradas:

In [ ]:
word_vectors.most_similar(positive=['airplane','flight'],negative=['ship'],topn=10)

[('plane', 0.6277297735214233),
 ('jet', 0.5784463882446289),
 ('flights', 0.5631440877914429),
 ('airliner', 0.5585241913795471),
 ('aircraft', 0.5546182990074158),
 ('jetliner', 0.550014853477478),
 ('NOTE_Expedia_Expedia.com', 0.5478827357292175),
 ('airplanes', 0.5451778173446655),
 ('Flight', 0.5407993197441101),
 ('airline', 0.5332231521606445)]

#  Desafio

 Reproduza os experimentos importando os *embeddings* treinados do Nilc disponíveis em: [http://nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc](http://nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc). Utilize vetores de tamanho 50 para não inviabilizar seus testes.